In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")

In [3]:
from langchain_groq import ChatGroq
model = ChatGroq(model="llama-3.3-70b-versatile",api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001AD1E7AE8F0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001AD1E7E0DC0>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
from langchain_core.messages import HumanMessage,SystemMessage

model.invoke(
    [HumanMessage(content="Hi, I am Avinash and i am a upcoming software engineer")]
)

AIMessage(content="Hi Avinash, it's great to meet you. Congratulations on pursuing a career in software engineering. That's an exciting and in-demand field. What areas of software engineering interest you the most? Are you specializing in any particular domain, such as web development, mobile app development, artificial intelligence, or something else?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 65, 'prompt_tokens': 49, 'total_tokens': 114, 'completion_time': 0.236363636, 'prompt_time': 0.002456039, 'queue_time': 0.049982261, 'total_time': 0.238819675}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_2ddfbb0da0', 'finish_reason': 'stop', 'logprobs': None}, id='run--7f3bd869-b506-44ff-9e2b-706cf98fa1ea-0', usage_metadata={'input_tokens': 49, 'output_tokens': 65, 'total_tokens': 114})

In [5]:
from langchain_core.messages import AIMessage
model.invoke([
    HumanMessage(content="Hi, I am Avinash and i am a upcoming software engineer"),
    AIMessage(content="Hello Avinash, it's nice to meet you. Congratulations on pursuing a career in software engineering. That's an exciting and in-demand field. What areas of software engineering interest you the most? Are you focusing on any specific programming languages, technologies, or industries? I'm here to help and chat about any topics related to software engineering."),
    HumanMessage("Hey! Tell me what's my name and what do i do?")
])

AIMessage(content="Your name is Avinash, and you're an upcoming software engineer.", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 142, 'total_tokens': 158, 'completion_time': 0.058181818, 'prompt_time': 0.185934214, 'queue_time': 0.050320066000000024, 'total_time': 0.244116032}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_2ddfbb0da0', 'finish_reason': 'stop', 'logprobs': None}, id='run--80f54c2a-6463-4649-bae6-650bc25b2151-0', usage_metadata={'input_tokens': 142, 'output_tokens': 16, 'total_tokens': 158})

### Message History
We can use message history class to wrap out model and make it stateful. This will keep track of input and output of model, and store them in a datastore

In [6]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()

    return store[session_id]

with_message_history = RunnableWithMessageHistory(model,get_session_history)

In [7]:
config = { "configurable":{"session_id":"chat1"}}

In [8]:
response = with_message_history.invoke([
    HumanMessage(content="Hi my name is Avinash and i am a full stack web developer trying to learn generative AI")
], config = config)

response.content

"Hello Avinash, nice to meet you. As a full stack web developer, you already have a strong foundation in programming and web development. Learning generative AI can be a fascinating and rewarding experience, and it's great that you're interested in exploring this field.\n\nGenerative AI is a subfield of artificial intelligence that focuses on generating new, synthetic data that resembles existing data. This can include images, videos, music, text, and more. Some popular applications of generative AI include:\n\n1. Image generation: Generating realistic images of faces, objects, and scenes.\n2. Text generation: Generating human-like text, such as articles, stories, and dialogue.\n3. Music generation: Generating music that sounds like it was composed by a human.\n4. Style transfer: Transferring the style of one image to another.\n\nTo get started with generative AI, you'll need to learn about some key concepts, including:\n\n1. Deep learning: A type of machine learning that uses neural n

In [9]:
response = with_message_history.invoke([
    HumanMessage("What is my name?"),
    
],config=config)

response.content

'Your name is Avinash.'

In [10]:
# Let's change the config --> change in session_id
config1 = { "configurable":{"session_id":"chat2"}}

response = with_message_history.invoke([
    HumanMessage("What is my name?"),
    
],config=config1)

response.content

"I don't know your name. I'm a large language model, I don't have the ability to know or recall personal information about individuals, including their names. Each time you interact with me, it's a new conversation and I don't retain any information from previous conversations. If you'd like to share your name with me, I'd be happy to address you by it, though!"

In [11]:
# Prompt templates
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful AI assistant whose main task is to explain the asked topic in simple words"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt | model 

In [12]:
chain.invoke({"messages":[HumanMessage("Tell me about database indexing")]})

AIMessage(content='**What is Database Indexing?**\n\nImagine you have a huge library with millions of books. Each book has a unique title, author, and publication date. When you want to find a specific book, you can either search through the entire library shelf by shelf or use a catalog that lists all the books in alphabetical order. The catalog helps you find the book quickly by providing a shortcut to its location.\n\nIn databases, **indexing** works similarly. It\'s a way to speed up the process of finding and retrieving specific data by creating a shortcut, or an "index," that points to the location of the data.\n\n**How Does Indexing Work?**\n\nWhen you create an index on a column in a database table, the database builds a separate data structure that contains the following:\n\n1. A copy of the column values\n2. A pointer to the location of the corresponding data in the table\n\nWhen you query the database, the index is used to quickly locate the required data. This reduces the t

In [13]:
with_message_history = RunnableWithMessageHistory(chain,get_session_history)

In [14]:
config3 = { "configurable":{"session_id":"chat3"}}

In [15]:
with_message_history.invoke(
    [
        HumanMessage("Hi my name is avinash")
    ],
    config=config3
)

AIMessage(content="Hello Avinash, it's nice to meet you. Is there something you'd like to know or talk about? I'm here to help and explain things in simple words. What's on your mind?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 60, 'total_tokens': 103, 'completion_time': 0.156363636, 'prompt_time': 0.019054428, 'queue_time': 0.055912964999999995, 'total_time': 0.175418064}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_e8c9074fa2', 'finish_reason': 'stop', 'logprobs': None}, id='run--d3411182-4726-4b07-b436-31bf6ca70bdd-0', usage_metadata={'input_tokens': 60, 'output_tokens': 43, 'total_tokens': 103})

In [16]:
# Adding more complexity to prompt

prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful AI assistant whose main task is to explain the asked topic in simple words. Explain the topics in {language}"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt | model 

In [17]:
res =  chain.invoke({
    "messages":[HumanMessage("Hi my name is avinash")],
    "language":"hindi"
})

In [18]:
res.content

'नमस्ते अविनाश, मैं आपका सहायक हूँ। मैं आपके प्रश्नों का उत्तर देने और आपको समझने में मदद करने के लिए यहाँ हूँ। क्या आप किसी विशेष विषय के बारे में जानना चाहते हैं या कोई प्रश्न पूछना चाहते हैं?'

In [19]:
with_message_history = RunnableWithMessageHistory(chain,get_session_history,input_messages_key="messages")

In [20]:
config4 = {"configurable":{"session_id":"chat4"}}

ans = with_message_history.invoke(
    {"messages":[
        HumanMessage("Hey I am avinash and i am a aspiring software developer"),
    ],
    "language":"hindi"},
    config=config4
)

In [21]:
ans

AIMessage(content='नमस्ते अविनाश, मैं आपका सहायक हूँ। मैं आपको सॉफ्टवेयर डेवलपमेंट से संबंधित जानकारी प्रदान करने में मदद करूँगा।\n\nसॉफ्टवेयर डेवलपमेंट एक बहुत ही रोचक और चुनौतीपूर्ण क्षेत्र है, जहाँ आप अपने विचारों और कल्पनाओं को वास्तविकता में बदल सकते हैं। यहाँ कुछ बातें हैं जो आपको सॉफ्टवेयर डेवलपमेंट की दुनिया में आगे बढ़ने में मदद कर सकती हैं:\n\n1. **प्रोग्रामिंग भाषाएँ**: सॉफ्टवेयर डेवलपमेंट में प्रोग्रामिंग भाषाएँ बहुत महत्वपूर्ण हैं। आप जावा, पायथन, सी++, जावास्क्रिप्ट आदि जैसी भाषाओं का अध्ययन कर सकते हैं।\n2. **वेब डेवलपमेंट**: वेब डेवलपमेंट में आप वेबसाइट्स, वेब एप्लिकेशन्स और मोबाइल एप्लिकेशन्स बना सकते हैं। आप एचटीएमएल, सीएसएस, जावास्क्रिप्ट आदि जैसी तकनीकों का उपयोग कर सकते हैं।\n3. **मोबाइल एप्लिकेशन डेवलपमेंट**: मोबाइल एप्लिकेशन डेवलपमेंट में आप एंड्रॉइड और आईओएस जैसे मोबाइल प्लेटफ़ॉर्म पर एप्लिकेशन्स बना सकते हैं।\n4. **डेटाबेस**: डेटाबेस में आप डेटा को संग्रहीत और प्रबंधित कर सकते हैं। आप एमएसएसक्यूएल, मायएसक्यूएल, मोंगोडीबी आदि जैसे डेटाबेस प्रबंधन प्रणालियों का उ

In [22]:
ans = with_message_history.invoke(
    {"messages":[
        HumanMessage("What do you know about me?"),
    ],
    "language":"english"},
    config=config4
)

In [23]:
ans.content

'अविनाश, मैं आपके बारे में निम्नलिखित जानकारी जानता हूँ:\n\n1. **नाम**: आपका नाम अविनाश है।\n2. **हित**: आप सॉफ्टवेयर डेवलपमेंट में रुचि रखते हैं और एक अस्पायरिंग सॉफ्टवेयर डेवलपर हैं।\n3. **भाषा**: आप अंग्रेजी भाषा समझते हैं और मुझसे बात करने के लिए इसका उपयोग करते हैं।\n4. **संचार**: आप मुझसे बात करने के लिए इस प्लेटफ़ॉर्म का उपयोग कर रहे हैं।\n\nयह जानकारी मुझे आपके द्वारा दी गई है, और मैं इसका उपयोग आपको बेहतर ढंग से समझने और आपकी मदद करने के लिए करता हूँ। अगर आपके पास कोई और जानकारी है जो आप मुझे देना चाहते हैं, तो मुझे बताएं!'

### Managing the conversation history
The list of messages can grow unbounded and can potentially overflow the llm context window, thats why it is needed to be managed.
An extra step is needed that limits the size of message we are passing in

In [31]:
from langchain_core.messages import SystemMessage,trim_messages

trimmer = trim_messages(
    max_tokens=60,
    strategy = "last",
    token_counter=model,
    include_system = True,
    allow_partial = False,
    start_on="human"
)

In [29]:
messages = [
    SystemMessage("You're a helpful AI assistant"),
    HumanMessage("Hi my name is avinash"),
    AIMessage("nice to meet you avinash, how can i help you"),
    HumanMessage("I like vannila icecream"),
    AIMessage("nice i like it too"),
    HumanMessage("What is 2+2?"),
    AIMessage("4")
]

In [32]:
trimmer.invoke(messages)

[SystemMessage(content="You're a helpful AI assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hi my name is avinash', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice to meet you avinash, how can i help you', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vannila icecream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice i like it too', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is 2+2?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={})]

In [33]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages = itemgetter("messages") | trimmer) |
    prompt |
    model
)

In [34]:
response = chain.invoke({
    "messages" : messages + [HumanMessage("What icecream do i like?")],
    "language":"English"
})

response.content

'You like vanilla icecream.'